In [1]:
from numba.core.errors import NumbaPerformanceWarning
import warnings

warnings.simplefilter('ignore', category=NumbaPerformanceWarning)

# Version 1: naive toy model
All pairs are produced randomly in the transverse plane, back-to-back, with periodic boundary conditions.

### Default simulation parameters for the Glasma and the Wong solvers

In [1]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
# L = 20      
# N = 1024    
L = 10      
N = 512 
tau_sim = 1     
DTS = 8     

# Glasma for Pb-Pb at 5.02 TeV
# su_group = 'su3'
# A = 207     
# sqrts = 5020        
# ns = 50     
# factor = 0.8        
# Qs = np.sqrt(0.13 * A**(1/3) * sqrts**0.25)        
# g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          
# mu = Qs / (g**2 * factor)          	
# ir = 0.1 * g**2 * mu         
# uv = 10.0       
 
# # Glasma
su_group = 'su3'
Qs = 2.0        
ns = 50    
factor = 0.8        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g2mu         
uv = 10.0              

# Wong
quark = 'charm'    
mass = 1.275       
tau_form = 1/(2*mass)*hbarc

tau_sim += tau_form

# tau_form = 0
initialization = 'toy'         
pT = 2    
ntp = 10**5  
nevents = 10    
representation = 'fundamental'      
boundary = 'periodic'       

# Results folder
folder = quark + '_pT_' + str(pT)       

# Store relevant parameters in a dictionary
p = {
    'QUARK': quark,
    'MASS': mass,           
    'TFORM': tau_form,   
    'NTP':  ntp,    
    'PT': pT,          
    'FOLDER': folder,         
    }

### Set environment variables

In [2]:
import os
os.environ["MY_NUMBA_TARGET"] = "cuda"
os.environ["PRECISION"] = "double"
os.environ["GAUGE_GROUP"] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

# Wong modules
from curraun import wong
wong.BOUNDARY = boundary
wong.WONG_TO_HOST = True
from curraun.wong import init_pos, init_charge, init_mom_toy
# init_mom_fonll
from curraun.particles_correlators import Angles

Using CUDA
Using SU(3)
Using double precision
Using double precision


### Simulation routine for evolving the Glasma & solving Wong's equations
Quark and antiquark pair initialized at the same position, with opposite momenta and random charge. Momenta $(p^\tau, p^x,p^y,p^\eta, p^z)$ 

In [3]:
import pickle
from tqdm import tqdm

current_path = os.getcwd() 
results_folder = 'results'
if not os.path.isdir(results_folder):
    os.makedirs(results_folder)
results_path = current_path + '/' + results_folder + '/'

In [4]:
def simulate(p, ev): 
    os.chdir(results_path)

    output = {}
    output['parameters'] = p.copy()

    # Derived parameters
    a = L/N
    E0 = N/L * hbarc
    DT = 1.0 / DTS
    formt = int(p['TFORM'] / a * DTS)
    maxt = int(tau_sim / a * DTS)

    # Initialize Glasma fields
    s = core.Simulation(N, DT, g)
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)

    # if initialization=='fonll':
    #     pTs_fonll, ntp_fonll = init_mom_fonll(p)
    #     ntp = ntp_fonll

    if initialization=='toy':
        # pT = p['PT'] / E0
        ntp = p['NTP']

    # Initialize the Wong solver
    wong_solver_q = wong.WongSolver(s, ntp)
    wong_solver_aq = wong.WongSolver(s, ntp)
    x0s_q, p0s_q, q0s_q = np.zeros((ntp, 3)), np.zeros((ntp, 5)), np.zeros((ntp, su.ALGEBRA_ELEMENTS))
    x0s_aq, p0s_aq, q0s_aq = np.zeros((ntp, 3)), np.zeros((ntp, 5)), np.zeros((ntp, su.ALGEBRA_ELEMENTS))
    masses = mass / E0 * np.ones(ntp)

    for i in range(ntp):
        if initialization=='toy':
            p0_q = init_mom_toy('pT', pT / E0)            
        # elif initialization=='fonll':
        #     pT_q = pTs_fonll[i] / E0
        #     p0_q = init_mom_toy('pT', pT_q / E0)
        p0_aq = [-p for p in p0_q]
        p0s_q[i, :], p0s_aq[i, :] = np.array(p0_q, dtype=object), np.array(p0_aq, dtype=object)

        x0_q, q0_q = init_pos(s.n), init_charge(representation)
        x0s_q[i, :], q0s_q[i, :] = x0_q, q0_q
        x0_aq, q0_aq = x0_q, init_charge(representation)
        x0s_aq[i, :], q0s_aq[i, :] = x0_aq, q0_aq
    
    wong_solver_q.initialize(x0s_q, p0s_q, q0s_q, masses)
    wong_solver_aq.initialize(x0s_aq, p0s_aq, q0s_aq, masses)

    # p_q, p_aq = np.zeros((maxt-formt, ntp, 5)), np.zeros((maxt-formt, ntp, 5))
    # active_q, active_aq = np.zeros((maxt-formt, ntp)), np.zeros((maxt-formt, ntp))
    qqbar_angle = Angles(wong_solver_q, wong_solver_aq, ntp)

    deta, dphi = np.zeros((maxt-formt, ntp)), np.zeros((maxt-formt, ntp))
    pTs = np.zeros((maxt-formt, ntp, 2))

    with tqdm(total=maxt) as pbar:
        for t in range(maxt):
            # Evolve Glasma fields
            core.evolve_leapfrog(s)

            # Solve Wong's equations
            if t>=formt:  
                qqbar_angle.compute()
                deta[t-formt] = qqbar_angle.deta.copy()
                dphi[t-formt] = qqbar_angle.dphi.copy()
                pTs[t-formt] = qqbar_angle.pT.copy() * E0
                
                wong_solver_q.evolve()
                # print(wong_solver_q.p.copy()*E0)
                # pmu_q = wong_solver_q.p.copy()
                # p_q[t-formt] = pmu_q
                # active_q[t-formt] = wong_solver_q.active.copy()
                # indices = np.argwhere(active_q[t-formt] == 0)

                wong_solver_aq.evolve()
                # print(wong_solver_aq.p.copy()*E0)
                # pmu_aq = wong_solver_aq.p.copy()
                # p_aq[t-formt] = pmu_aq
                # active_aq[t-formt] = wong_solver_aq.active.copy()

            pbar.set_description("Event " + str(ev+1))
            pbar.update(1)

    # output['p_q'], output['p_aq'] = p_q, p_aq
    # output['active_q'], output['active_aq'] = active_q, active_aq

    tau = np.linspace(0, tau_sim-p['TFORM'], maxt-formt)
    output['tau'] = tau
    output['deta'], output['dphi'], output['pTs'] = deta, dphi, pTs

    wong_folder = p['FOLDER']
    if not os.path.isdir(wong_folder):
        os.makedirs(wong_folder)
    wong_path = results_path + '/' + wong_folder + '/'
    os.chdir(wong_path)

    filename = 'event_' + str(ev+1) + '.pickle'
    with open(filename, 'wb') as handle:
        pickle.dump(output, handle)
    return output

### Running the Wong solver over multiple Glasma events

In [5]:
# quarks = ['jet']
# quark_masses = [1]
# pTs = [100]
# formation_times = [0]

# quarks = ['beauty']
# quark_masses = [4.18]
# pTs = [2]
# formation_times = [0.02]

quarks = [quark]
quark_masses = [mass]
pTs = [pT]
formation_times = [tau_form]

for iq in range(len(quarks)):
    print(quarks[iq].capitalize() + " quark")
    p['QUARK'], p['MASS'] = quarks[iq], quark_masses[iq]
    # p['TFORM'] = 1/(2*p['MASS'])*hbarc 
    p['TFORM'] = formation_times[iq]

    for pT in pTs:
        print('Transverse momentum', pT, 'GeV')
        p['PT'] = pT
        tag = quarks[iq] + '_pT_' + str(pT)
        p['FOLDER'] = 'corr_toy_pT_' + str(pT) + '_' + quarks[iq] 

        for ev in range(nevents):
            simulate(p, ev)

Charm quark
Transverse momentum 2 GeV


# Version 2: border toy model

### Simulation routine for evolving the Glasma & solving Wong's equations
Quark and antiquark pair initialized at the same position, with opposite momenta and random charge. at the boundary of the Glasma.If they exit the Glasma region, their dynamics is frozen.  Momenta $(p^x,p^y,p^z,p^\eta)$ and the ```active``` array, which keeps track of particles which reached the boundary, are stored throughout the evolution.

In [ ]:
from curraun.particles_correlators import BorderAngles

def init_mom_toy_in(pT):
    """
        Initialize all particles with the same initial transverse momentum
        but the angle is in (0, pi)
    """
    # angle = np.pi*np.random.rand(1) - np.pi/2
    angle = np.pi*np.random.rand(1)
    p0 = [0.0, pT * np.cos(angle), pT * np.sin(angle), 0.0, 0.0]

    return p0

def init_pos_in(n):

    x = np.random.rand(1) * n
    x0 = [x, 0.0, 0.0]

    return x0

In [ ]:
def simulate(p, ev): 
    os.chdir(results_path)

    # Derived parameters
    a = L/N
    E0 = N/L * hbarc
    DT = 1.0 / DTS
    formt = int(p['TFORM'] / a * DTS)
    maxt = int(tau_sim / a * DTS)

    # Initialize Glasma fields
    s = core.Simulation(N, DT, g)
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)

    # if initialization=='fonll':
    #     pTs_fonll, ntp_fonll = init_mom_fonll(p)
    #     ntp = ntp_fonll
    #     p['NTP'] = ntp_fonll

    if initialization=='toy':
        pT = p['PT'] 
        ntp = p['NTP']

    output = {}
    output['parameters'] = p.copy()

    # Initialize the Wong solver
    wong_solver = wong.WongSolver(s, ntp)
    x0s, p0s, q0s = np.zeros((ntp, 3)), np.zeros((ntp, 5)), np.zeros((ntp, su.ALGEBRA_ELEMENTS))
    masses = mass / E0 * np.ones(ntp)

    for i in range(ntp):
        if initialization=='toy':
            p0 = init_mom_toy_in(pT / E0)   
        # elif initialization=='fonll':
        #     pT = pTs_fonll[i]
        #     p0 = init_mom_toy_in(pT / E0)
        p0s[i, :] = np.array(p0, dtype=object)

        x0, q0 = init_pos_in(s.n), init_charge(representation)
        x0s[i, :], q0s[i, :] = np.array(x0, dtype=object), np.array(q0, dtype=object)
    
    wong_solver.initialize(x0s, p0s, q0s, masses)
    qqbar_angle = BorderAngles(wong_solver, ntp, p0s)

    deta, dphi = np.zeros((maxt-formt, ntp)), np.zeros((maxt-formt, ntp))
    pTs = np.zeros((maxt-formt, ntp))

    with tqdm(total=maxt) as pbar:
        for t in range(maxt):
            # Evolve Glasma fields
            core.evolve_leapfrog(s)

            # Solve Wong's equations
            if t>=formt: 
                qqbar_angle.compute()

                deta[t-formt] = qqbar_angle.deta.copy()
                dphi[t-formt] = qqbar_angle.dphi.copy()
                pTs[t-formt] = qqbar_angle.pT.copy() * E0

                # angles[t-formt] = qqbar_angle.angle.copy()
                # pTs[t-formt] = qqbar_angle.pT.copy() * E0

                wong_solver.evolve()

            pbar.set_description("Event " + str(ev+1))
            pbar.update(1)
    
    output['p0'] = p0
    tau = np.linspace(p['TFORM'], tau_sim, maxt-formt)
    output['tau'] = tau
    output['deta'], output['dphi'], output['pTs'] = deta, dphi, pTs

    wong_folder = p['FOLDER']
    if not os.path.isdir(wong_folder):
        os.makedirs(wong_folder)
    wong_path = results_path + '/' + wong_folder + '/'
    os.chdir(wong_path)

    filename = 'event_' + str(ev+1) + '.pickle'
    with open(filename, 'wb') as handle:
        pickle.dump(output, handle)

    os.chdir(current_path)

In [ ]:
quarks = ['jet']
quark_masses = [1]
pTs = [100]
formation_times = [0]

for iq in range(len(quarks)):
    print(quarks[iq].capitalize() + " quark")
    p['QUARK'], p['MASS'] = quarks[iq], quark_masses[iq]
    # p['TFORM'] = 1/(2*p['MASS'])*hbarc 
    p['TFORM'] = formation_times[iq]

    for pT in pTs:
        print('Transverse momentum', pT, 'GeV')
        p['PT'] = pT
        tag = quarks[iq] + '_pT_' + str(pT)
        p['FOLDER'] = 'corr_toy_border_pT_' + str(pT) + '_' + quarks[iq] 

        for ev in range(nevents):
            simulate(p, ev)